In [5]:
import pandas as pd
from darts.models import AutoARIMA
from darts import TimeSeries
import pickle


inflation_df = pd.read_csv("Inflation.csv", index_col=0, header = [0,1])
CPI_df = pd.read_csv("CPI.csv", index_col=0, header = [0,1])


inflation_df.columns = inflation_df.columns.droplevel(1)
cols = inflation_df.columns.values  # Get column names as a NumPy array

# Rename only specific indexed columns
cols[-12] = "Global"

inflation_df.columns = cols


inflation_df.index = pd.to_datetime(inflation_df.index.astype(str), format='%Y%m')

inflation_df = inflation_df.iloc[2:].asfreq("MS")

inflation_df_train = inflation_df[inflation_df.index < pd.Timestamp('2000-03-01')]
inflation_df_test = inflation_df[inflation_df.index >= pd.Timestamp('2000-03-01')]

inflation_series = TimeSeries.from_dataframe(inflation_df)

country_names = inflation_df.columns[:-12]


In [6]:
len(inflation_df.columns[:-12])

91

In [7]:
def select_covariates(inflation_df, country, p = 4):
    # Step 1: Select the target variable (inflation) for the given country
    target = inflation_df[country]
    
    # Step 2: Create a copy of inflation_df for manipulation (we won't modify the original)
    selected_covariates = pd.DataFrame()
    
    # Step 3: Generate autoregressive lags (p lags) for the selected country without modifying the original dataframe
    for i in range(1, p + 1):
        selected_covariates[f'{country}_lag_{i}'] = inflation_df[country].shift(i)
    
    # Step 4: Select inflation data for all other countries (excluding the selected country)
    other_countries_inflation = inflation_df[country_names].drop(columns=[country]).shift(1)  # Drop the selected country
    
    # Step 5: Select the regional inflation factor for the selected country
    # Assuming that you have a column with regional factors, e.g., 'region' column in the dataset
    global_factor = inflation_df["Global"].shift(1)

    # Step 6: Add monthly dummy variables for seasonality
    monthly_dummies = inflation_df[inflation_df.columns[-11:]]
    
    # Step 7: Combine all the covariates into the selected_covariates DataFrame
    selected_covariates = pd.concat([selected_covariates, 
                                     other_countries_inflation, 
                                     global_factor,
                                     monthly_dummies], axis=1)
    
    return selected_covariates


# Article baselines

## AR baseline (and RW)

In [3]:
def random_walk(series: TimeSeries, horizon: int) -> TimeSeries:
    return series.append(series.shift(horizon))

def seasonal_random_walk(series: TimeSeries, horizon: int) -> TimeSeries:
    return series.append(series.shift(12))

In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import ar_select_order

# Assuming `inflation_series` is a DataFrame with countries as columns
forecasts_country = {}
start_date = pd.Timestamp('2000-03-01')
forecast_horizon = 1
train_length = 240  # Rolling window size

for i, country_name in enumerate(country_names):
    print(f"{i}/{len(country_names)}", end = "\r", flush=True)
    
    # Extract the target time series
    target_series = inflation_df[country_name]
    historical_forecasts = []

    # Get the past covariates (excluding the target country itself)
    covariate_cols = inflation_df.columns[-11:]
    if country_name in covariate_cols:
        covariate_cols.remove(country_name)  # Avoid using the target as its own covariate

    covariates = inflation_df[covariate_cols]

    # Define the starting index
    start_idx = target_series.index.get_loc(start_date)

    for t in range(start_idx, len(target_series) - forecast_horizon):
        train_end_idx = t
        train_start_idx = max(0, train_end_idx - train_length)

        # Extract training data
        train_series = target_series.iloc[train_start_idx:train_end_idx + 1]
        train_covariates = covariates.iloc[train_start_idx:train_end_idx + 1]

        # Fit ARIMA model with exogenous regressors
        res_model = ar_select_order(train_series, maxlag=12, exog=train_covariates)
        model_fit = res_model.model.fit()

        # Prepare exogenous variables for forecasting
        forecast_exog = covariates.iloc[[train_end_idx + forecast_horizon]]

        # Forecast one step ahead
        forecast = model_fit.forecast(steps=forecast_horizon, exog=forecast_exog)
        historical_forecasts.append(forecast.iloc[0])

    forecasts_country[country_name] = pd.Series(historical_forecasts, 
                                                index=target_series.index[start_idx:-forecast_horizon])
    
with open('AR_all_forecast.pkl', 'wb') as f:
    pickle.dump(forecasts_country, f)
    

KeyboardInterrupt: 

In [57]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import ar_select_order

# Assuming `inflation_series` is a DataFrame with countries as columns
forecasts_country = {}
start_date = pd.Timestamp('2000-03-01')
forecast_horizon = 1
train_length = 240  # Rolling window size

for i, country_name in enumerate(country_names):
    print(f"{i}/{len(country_names)}", end = "\r", flush=True)
    
    # Extract the target time series
    target_series = inflation_df[country_name]
    historical_forecasts = []

    # Get the past covariates (excluding the target country itself)
    covariate_cols = inflation_df.columns[-11:]
    if country_name in covariate_cols:
        covariate_cols.remove(country_name)  # Avoid using the target as its own covariate

    covariates = inflation_df[covariate_cols].astype(int)

    covariates["Factor"] = inflation_df[region_dict[country_name]].shift(1).fillna(0)
    covariates["Global Factor"] = inflation_df["Global"].shift(1).fillna(0)

    # Define the starting index
    start_idx = target_series.index.get_loc(start_date)

    for t in range(start_idx, len(target_series) - forecast_horizon):
        train_end_idx = t
        train_start_idx = max(0, train_end_idx - train_length)

        # Extract training data
        train_series = target_series.iloc[train_start_idx:train_end_idx + 1]
        train_covariates = covariates.iloc[train_start_idx:train_end_idx + 1]

        # Fit ARIMA model with exogenous regressors
        res_model = ar_select_order(train_series, maxlag=12, exog=train_covariates)
        model_fit = res_model.model.fit()

        # Prepare exogenous variables for forecasting
        forecast_exog = covariates.iloc[[train_end_idx + forecast_horizon]]

        # Forecast one step ahead
        forecast = model_fit.forecast(steps=forecast_horizon, exog=forecast_exog)
        historical_forecasts.append(forecast.iloc[0])

    forecasts_country[country_name] = pd.Series(historical_forecasts, 
                                                index=target_series.index[start_idx:-forecast_horizon])

with open('CM_all_forecast.pkl', 'wb') as f:
    pickle.dump(forecasts_country, f)

In [59]:
from sklearn.metrics import root_mean_squared_error as rmse
import numpy as np

with open('CM_all_forecast.pkl', 'rb') as f:
    cm_forecasts = pickle.load(f)


target_df = inflation_df.loc["2000-03-01":"2019-11-01"]

np.mean([rmse(target_df[country_name], cm_forecasts[country_name]) for country_name in country_names])

0.8295975751572828

## Random Forest

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from math import sqrt, log2
import tqdm
import shap

# Set parameters
start_date = pd.Timestamp('2000-03-01')
train_length = 240
max_horizon = 12

# Helper function for feature grid
def get_predictor_grid(p):
    return list(set([p, int(2/3 * p), int(1/3 * p), int(sqrt(p)), int(log2(p))]))

p_values = [4, 12]  # Values of lags to try

for i, country_name in enumerate(country_names):
    print(f"{i}/{len(country_names)}: {country_name}", flush=True)

    target_series = inflation_df[country_name]
    start_idx = target_series.index.get_loc(start_date)

    for h in range(1, max_horizon + 1):
        print(h)
        historical_forecasts = []
        best_params_list = []
        shap_values = []

        for t in tqdm.tqdm(range(start_idx, len(target_series) - h)):
            train_end_idx = t
            train_start_idx = max(0, train_end_idx - train_length)

            train_data = inflation_df.iloc[train_start_idx:train_end_idx + 1]
            target_shifted = train_data[country_name].shift(-h)

            # Forecast covariates for time t+h
            full_forecast_data = inflation_df.iloc[train_start_idx:train_end_idx + h + 1]

            if t % 120 == 0 or t == start_idx:
                best_score = float('inf')
                for p in p_values:
                    covariates = select_covariates(train_data, country_name, p=p).iloc[p:-h]
                    target = target_shifted.iloc[p:-h]
                    X_tmp = select_covariates(full_forecast_data, country_name, p=p).iloc[[-1]]

                    if len(target) < 10:
                        continue  # skip too-short windows

                    param_grid = {
                        'max_features': get_predictor_grid(len(covariates.columns))
                    }

                    rf_model = RandomForestRegressor(
                        random_state=42, n_estimators=500, min_samples_leaf=5, n_jobs=-1
                    )
                    tscv = TimeSeriesSplit(n_splits=5)
                    grid_search = GridSearchCV(rf_model, param_grid, cv=tscv,
                                               scoring='neg_mean_squared_error', verbose=0)
                    grid_search.fit(covariates, target)
                    score = -grid_search.best_score_

                    if score < best_score:
                        best_score = score
                        best_model = grid_search.best_estimator_
                        best_p = p
                        X_forecast = X_tmp
                    best_params_list.append({'best_p': best_p,
                                             'max_features': best_model.max_features})
            else:
                covariates = select_covariates(train_data, country_name, p=best_p).iloc[best_p:-h]
                target = target_shifted.iloc[best_p:-h]
                X_forecast = select_covariates(full_forecast_data, country_name, p=best_p).iloc[[-1]]
                best_model.fit(covariates, target)

            forecast = best_model.predict(X_forecast)
            # Initialize explainer
            shap_values.append(shap.TreeExplainer(best_model).shap_values(X_forecast))
            # Calculate SHAP values on training covariates
            forecast_index = target_series.index[t + h]
            historical_forecasts.append((forecast_index, forecast[0]))
        out = {"forecast": historical_forecasts,
               "best_params": best_params_list,
               "shap_values": shap_values}
        with open(f'RF_forecasts/RF_forecast_h{h}_{country_name}.pkl', 'wb') as f:
            pickle.dump(historical_forecasts, f)

0/91: Austria
1


100%|██████████| 237/237 [06:27<00:00,  1.63s/it]


2


100%|██████████| 236/236 [06:21<00:00,  1.62s/it]


3


...

100%|██████████| 228/228 [05:44<00:00,  1.51s/it]


11


100%|██████████| 227/227 [05:03<00:00,  1.34s/it]


12


100%|██████████| 226/226 [04:58<00:00,  1.32s/it]


In [52]:
best_p

In [92]:
forecasts_country["Austria"]

time
2000-04-01    0.114470
2000-05-01    0.121995
2000-06-01    0.170564
2000-07-01    0.404788
2000-08-01    0.345790
                ...   
2019-08-01    0.118796
2019-09-01    0.331449
2019-10-01    0.043328
2019-11-01    0.114938
2019-12-01    0.133400
Length: 237, dtype: float64

In [76]:
train_data["Europe"]

time
1980-03-01    0.764569
1980-04-01    1.321007
1980-05-01    0.576158
1980-06-01    0.530069
1980-07-01    0.923869
                ...   
1999-11-01    0.195006
1999-12-01    0.382021
2000-01-01   -0.029050
2000-02-01    0.332836
2000-03-01    0.567240
Freq: MS, Name: Europe, Length: 241, dtype: float64

In [48]:
import pickle

with open("RF_all_forecast.pkl", "rb") as f:
    rf_forecasts = pickle.load(f)

In [50]:
rf_forecasts["Austria"]

time
2000-03-01    0.243202
2000-04-01    0.022444
2000-05-01    0.103281
2000-06-01    0.412508
2000-07-01    0.248727
                ...   
2019-07-01   -0.218793
2019-08-01    0.107846
2019-09-01    0.406583
2019-10-01    0.112337
2019-11-01    0.138207
Freq: MS, Length: 237, dtype: float64

In [95]:
from sklearn.metrics import root_mean_squared_error as rmse
import numpy as np

target_df = inflation_df.loc["2000-03-01":"2019-11-01"]

np.mean([rmse(target_df[country_name], rf_forecasts[country_name]) for country_name in country_names])

print(rmse(target_df["Austria"], rf_forecasts["Austria"]))
print(rmse(target_df["Austria"], forecasts_country["Austria"]))

0.2018622536551119
0.34092707765020175


# Dlinear

In [ ]:
import numpy as np
import pandas as pd
from darts.models import DLinearModel
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from math import sqrt, log2
import tqdm
import logging
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
from darts.metrics import rmse

dlinear = DLinearModel(

    input_chunk_length=4,

    output_chunk_length=1,

    kernel_size=12,

    batch_size = 8,

    n_epochs=20,
)

param_grid = {'kernel_size': [5, 9, 13],
              'input_chunk_length': [4, 8, 12, 24],
              "output_chunk_length": [1],
              "pl_trainer_kwargs": [{"enable_progress_bar": False, "enable_model_summary": False}]
              }

# Assuming `inflation_series` is a DataFrame with countries as columns
forecasts_country = {}
start_date = pd.Timestamp('2000-03-01')
train_length = 240  # Rolling window size
for h in [1, 3, 6, 12]:
    print(f"h: {h}")
    dlinear = DLinearModel(

    input_chunk_length=4,

    output_chunk_length=h,

    kernel_size=12,

    batch_size = 8,

    n_epochs=20)

    # Extract the target time series
    historical_forecasts = []
    # Define the starting index
    start_idx = inflation_df.index.get_loc(start_date)
    for t in tqdm.tqdm(range(start_idx, len(inflation_df) - 1 - h)):
        train_end_idx = t
        train_start_idx = max(0, train_end_idx - train_length)
        train_data = inflation_series[train_start_idx:train_end_idx + 1]
        # Extract training data
        train_series = train_data[4:]
        if inflation_series[t].time_index.month == 12 or t == start_idx:
            # Fit dlinear model with grid search
            # Use TimeSeriesSplit for time series cross-validation
            grid_search = dlinear.gridsearch(param_grid, 
                                            train_series[list(country_names)], 
                                            inflation_series.drop_columns(list(country_names)), 
                                            forecast_horizon = len(train_series)//5, stride = len(train_series)//5, 
                                            verbose=False, metric=rmse, show_warnings = False)
            # Get the best model from grid search
            best_model = grid_search[0]
            best_model.fit(series=train_series[list(country_names)], past_covariates=inflation_series.drop_columns(list(country_names)))
        else:
            best_model.fit(series=train_series[list(country_names)], past_covariates=inflation_series.drop_columns(list(country_names)))
        # Forecast one step ahead
        forecast = best_model.predict(h)

        historical_forecasts.append(forecast[0])
    forecasts = pd.Series(historical_forecasts)

    with open(f'dlinear_forecasts/dlinear_forecast_h{h}.pkl', 'wb') as f:
        pickle.dump(forecasts, f)

h: 1


  0%|          | 0/236 [00:00<?, ?it/s]


TypeError: BaseForest.fit() got an unexpected keyword argument 'series'

In [ ]:
import shap
best_model.fit(series=train_series[list(country_names)], past_covariates=inflation_series.drop_columns(list(country_names)))
shap.DeepExplainer(best_model.model, best_model.pred_loader)



best_model.model

Attempting to retrain/fine-tune the model without resuming from a checkpoint. This is currently discouraged. Consider model `DLinearModel.load_weights()` to load the weights for fine-tuning.


In [19]:
with open("dlinear_all_forecast.pkl", "rb") as f:
    dlinear_forecasts = pickle.load(f)

In [23]:
forecasts = dlinear_forecasts[0]
for res in dlinear_forecasts[1:]:
    forecasts = forecasts.append(res)

forecasts

<TimeSeries (DataArray) (time: 237, component: 91, sample: 1)> Size: 173kB
array([[[-0.41839082],
        [ 4.54192656],
        [ 0.53569259],
        ...,
        [ 0.46626565],
        [ 5.27895799],
        [ 1.15521379]],

       [[ 0.75697159],
        [-0.21616837],
        [-0.18099225],
        ...,
        [-0.05458556],
        [-2.3016431 ],
        [ 0.76079619]],

       [[-0.0994057 ],
        [ 3.71126542],
        [ 0.58068289],
        ...,
...
        ...,
        [-0.36822091],
        [ 1.61492255],
        [ 0.12836746]],

       [[ 0.30103833],
        [ 4.53557298],
        [ 0.43600446],
        ...,
        [-0.02300835],
        [-3.60538598],
        [ 0.2344769 ]],

       [[ 0.06054387],
        [-3.08486527],
        [ 0.2082653 ],
        ...,
        [-0.10849075],
        [ 1.05452229],
        [ 0.61728527]]])
Coordinates:
  * time       (time) datetime64[ns] 2kB 2000-04-01 2000-05-01 ... 2019-12-01
  * component  (component) object 728B 'Austria' 'Burundi' ... 'South Africa'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [29]:
target = inflation_series[pd.Timestamp('2000-03-01'):][list(country_names)]

In [30]:
rmse(target, forecast)

0.6384034374915464